In [1]:
FEATURES = ["dom_x", "dom_y", "dom_z", "dom_time", "charge"]  # (x,y,z,t,q)

In [2]:
from typing import Dict, Callable
import torch

In [3]:
import sys, os

# go two levels up:  .../Graphnet-Applications
repo_root = os.path.abspath("../..")
sys.path.append(repo_root)
from MyClasses.detector_pone import PONE

graphnet [MainProcess] WARNING  2026-01-11 06:25:09 - has_jammy_flows_package - `jammy_flows` not available. Normalizing Flow functionality is missing.


In [4]:
det = PONE(
    replace_with_identity=["charge", "dom_time", "dom_x", "dom_y", "dom_z"]
)

graphnet [MainProcess] INFO     2026-01-11 06:25:12 - PONE.__init__ - Writing log to logs/graphnet_20260111-062512.log


In [5]:
from graphnet.data.dataset.parquet.parquet_dataset import ParquetDataset
from graphnet.models.detector.prometheus  import  Prometheus
from graphnet.models.graphs  import  KNNGraph
from graphnet.models.graphs.nodes  import  NodesAsPulses

FEATURES = ["dom_x", "dom_y", "dom_z", "dom_time", "charge"]  # (x,y,z,t,q)

data_representation = KNNGraph(
    detector=PONE(),
    node_definition=NodesAsPulses(),
    nb_nearest_neighbours=8,
    distance_as_edge_feature = True
)

dataset = ParquetDataset(
    path="/project/def-nahee/kbas/POM_Response_Parquet/merged" ,
    pulsemaps="features",
    truth_table="truth",
    features=FEATURES,
    truth=["energy"],
    data_representation = data_representation,
)



graphnet [MainProcess] WARNING  2026-01-11 06:25:20 - has_icecube_package - `icecube` not available. Some functionality may be missing.
graphnet [MainProcess] WARNING  2026-01-11 06:25:20 - has_km3net_package - `km3net` not available. Some functionality may be missing.
graphnet [MainProcess] WARNING  2026-01-11 06:25:20 - <module> - `graphnet.models.graphs` will be depricated soon. All functionality has been moved to `graphnet.models.data_representation`.


In [6]:
from graphnet.data.dataloader import DataLoader

In [7]:
loader = DataLoader(
    dataset,
    batch_size=4,
    shuffle=True,
    num_workers=4,
    multiprocessing_context="spawn",  
    persistent_workers=True,         
)


# datasetleri onceden ayirip ayri ayri koysan iyi olcak gibi. merged icine train, test, val koy

In [8]:
it = iter(loader)


graphnet [SpawnProcess-1] WARNING  2026-01-11 06:25:59 - has_icecube_package - `icecube` not available. Some functionality may be missing.
graphnet [SpawnProcess-1] WARNING  2026-01-11 06:25:59 - has_km3net_package - `km3net` not available. Some functionality may be missing.
graphnet [SpawnProcess-1] WARNING  2026-01-11 06:26:03 - has_jammy_flows_package - `jammy_flows` not available. Normalizing Flow functionality is missing.
graphnet [SpawnProcess-2] WARNING  2026-01-11 06:26:04 - has_icecube_package - `icecube` not available. Some functionality may be missing.
graphnet [SpawnProcess-2] WARNING  2026-01-11 06:26:05 - has_km3net_package - `km3net` not available. Some functionality may be missing.
graphnet [SpawnProcess-2] WARNING  2026-01-11 06:26:09 - has_jammy_flows_package - `jammy_flows` not available. Normalizing Flow functionality is missing.
graphnet [SpawnProcess-3] WARNING  2026-01-11 06:26:10 - has_icecube_package - `icecube` not available. Some functionality may be missing.

In [22]:
batch = next(it)


In [12]:
from graphnet.models.gnn  import  DynEdge
backbone = DynEdge(nb_inputs=5, nb_neighbours=8, 
                      # features_subset 
                      # dynedge_layer_sizes 
                      # post_processing_layer_sizes 
                      # readout_layer_sizes
                      global_pooling_schemes = ["min","max","mean","sum"],
                      add_global_variables_after_pooling = True, 
                      add_norm_layer = False, 
                      skip_readout = False                      
                     )

In [24]:
model = backbone(batch)

In [25]:
from graphnet.models.task import StandardLearnedTask
from graphnet.utilities.maths import eps_like
from torch import Tensor

In [26]:
def logarithm(E: torch.Tensor) -> torch.Tensor:
    """
    Forward transform: E -> log10(E)
    Safety: clamps E to >= eps to avoid log10(0) / log10(negative).
    """
    E_safe = torch.clamp(E, min=eps_like(E))
    return torch.log10(E_safe)

def exponential(t: torch.Tensor) -> torch.Tensor:
    """
    Inverse transform: log10(E) -> E
    """
    return torch.pow(10.0, t)

interval = (1e1, 1e8) 


In [27]:
class DepositedEnergyLog10Task(StandardLearnedTask):
    default_target_labels = ["energy"]
    default_prediction_labels = ["log10_energy_pred"]
    nb_inputs = 1

    def _forward(self, x: Tensor) -> Tensor:
        return x[:, :1]

   

In [28]:
from graphnet.training.loss_functions import LogCoshLoss

In [29]:
task = DepositedEnergyLog10Task(
    hidden_size = backbone.nb_outputs,
    loss_function = LogCoshLoss(),
    target_labels =["energy"],
    prediction_labels = ["log_energy"],
    transform_prediction_and_target = None,
    transform_target = logarithm,
    transform_inference = exponential,
    transform_support = interval,
    loss_weight = None)

# _validate_and_set_transforms calistirmana gerek yok. init esnasinda calistitiliyor.

In [31]:
pred = task(model)

In [32]:
pred

tensor([[91227.9219],
        [87541.8516],
        [88054.9922],
        [77645.0547]], grad_fn=<SliceBackward0>)

In [33]:
task.compute_loss(pred, batch)

tensor(86113.4566, dtype=torch.float64, grad_fn=<AddBackward0>)